In [ ]:
# RBT Superconductor Analysis Dashboard
## Interactive Tc vs Bond Length Analysis

This notebook provides interactive visualization tools for analyzing 2-D superconductor candidates using the RBT (Replication-Based Theory) framework.

### Key Features:
- Parity analysis (K = 0 test)
- Bond quantum energy estimation (ħω*/π)
- Predicted Tc vs bond length relationships
- Interactive structure visualization

### RBT Criteria:
1. **Parity condition**: K = 0 (even number of odd-degree vertices)
2. **Energy condition**: ħω*/π ≥ 0.081 eV
3. **Expected Tc**: T_c ∝ (ħω*/π)^2 for successful candidates


In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Add scripts directory to path
sys.path.append('.')

# Import our RBT analysis tools
from parity_check import RBTParityChecker
from bond_quantum import BondQuantumAnalyzer

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🚀 RBT Analysis Dashboard Loaded")
print("Ready to analyze 2-D superconductor candidates!")


In [ ]:
## 1. Batch Analysis of Example Structures


In [ ]:
def analyze_structure_batch(cif_directory="../data/example_CIFs/"):
    """
    Analyze all CIF files in a directory and compile results.
    """
    # Initialize analyzers
    parity_checker = RBTParityChecker()
    bond_analyzer = BondQuantumAnalyzer()
    
    results = []
    
    # Find all CIF files
    cif_files = [f for f in os.listdir(cif_directory) if f.endswith('.cif')]
    
    print(f"Found {len(cif_files)} CIF files to analyze...")
    
    for cif_file in cif_files:
        filepath = os.path.join(cif_directory, cif_file)
        print(f"\nAnalyzing: {cif_file}")
        
        try:
            # Parity analysis
            parity_results = parity_checker.analyze_structure(filepath)
            
            # Bond quantum analysis  
            bond_results = bond_analyzer.analyze_structure(filepath)
            
            # Compile results
            result = {
                'file': cif_file,
                'formula': parity_results['formula'],
                'K_parity': parity_results['K_parity'],
                'parity_pass': parity_results['passes_parity_test'],
                'phonon_energy_meV': bond_results['phonon_energy_meV'],
                'phonon_energy_eV': bond_results['phonon_energy_eV'],
                'energy_pass': bond_results['passes_energy_test'],
                'both_pass': (parity_results['passes_parity_test'] and 
                             bond_results['passes_energy_test']),
                'shortest_bond_distance': (bond_results['shortest_bond']['distance'] 
                                         if bond_results['shortest_bond'] else None),
                'shortest_bond_type': (bond_results['shortest_bond']['bond_type'] 
                                     if bond_results['shortest_bond'] else None)
            }
            
            results.append(result)
            
        except Exception as e:
            print(f"  ❌ Error analyzing {cif_file}: {e}")
            continue
    
    return pd.DataFrame(results)

# Run batch analysis
df_results = analyze_structure_batch()
print(f"\n✅ Analysis complete! {len(df_results)} structures analyzed.")
